In [21]:
from copy import deepcopy

import numpy as np
import dummypydoocs as pydoocs
from scipy.optimize import Bounds, minimize
import sympy

from environments import ARESEAOptimization

In [4]:
senv = ARESEAOptimization(backend="simulation", backendargs={"measure_beam": "direct"})
menv = ARESEAOptimization(backend="machine", backendargs={"measure_beam": "us"})

In [18]:
energy = 154e6

## Compute incoming beam

In [19]:
menv.next_initial = np.zeros(5)
menv.reset()
beam_parameters = menv.backend.compute_beam_parameters()

print(f"Actuators are set to {menv.backend.actuators}")
print(f"Measured beam parameters: {beam_parameters}")

Actuators are set to [0. 0. 0. 0. 0.]
Measured beam parameters: [-0.00155367 -0.00029363  0.00128563  0.00067329]


In [76]:
senv.next_initial = np.zeros(5)
senv.reset()

segment = deepcopy(senv.backend._segment)
segment.AREABSCR1.is_active = False
R = segment.transfer_map(energy)

In [81]:
R_inv = np.linalg.inv(R)

mu_out = np.array([beam_parameters[0], 0, beam_parameters[1], 0, 0, 0, 1])
mu_in = np.matmul(R_inv, mu_out)

cov_out = np.array([
    [eam_parameters[1]**2,           0,          0,           0,          0,          0, 0],
    [                   0, sigma_xp**2,          0,           0,          0,          0, 0],
    [                   0,           0, sigma_y**2,       cor_y,          0,          0, 0],
    [                   0,           0,      cor_y, sigma_yp**2,          0,          0, 0],
    [                   0,           0,          0,           0, sigma_s**2,      cor_s, 0],
    [                   0,           0,          0,           0,      cor_s, sigma_p**2, 0],
    [                   0,           0,          0,           0,          0,          0, 0]

])

array([-1.5536664e-03,  0.0000000e+00, -2.9362800e-04,  0.0000000e+00,
        0.0000000e+00,  0.0000000e+00,  1.0000000e+00])

In [68]:
mu0 = sympy.MatrixSymbol("mu_0", 7, 1)
mu1 = sympy.MatrixSymbol("mu_1", 7, 1)
R = sympy.MatrixSymbol("R", 7, 7)
mueq = sympy.Eq(mu1, sympy.MatMul(R, mu0), evaluate=False)
mueq

Eq(mu_1, R*mu_0)

In [75]:
mueq = sympy.Eq(sympy.MatMul(R**(-1), mu1), sympy.MatMul(R**(-1), R, mu0), evaluate=False)
mueq

Eq(R**(-1)*mu_1, R**(-1)*R*mu_0)

In [71]:
cov0 = sympy.MatrixSymbol("C_0", 7, 7)
cov1 = sympy.MatrixSymbol("C_1", 7, 7)
R = sympy.MatrixSymbol("R", 7, 7)
coveq = sympy.Eq(cov1, sympy.MatMul(R, cov0, R.T), evaluate=False)
coveq

Eq(C_1, R*C_0*R.T)

In [73]:
coveq = sympy.Eq(sympy.MatMul(R**(-1), cov1, R.T**(-1)), sympy.MatMul(R**(-1), R, cov0, R.T, R.T**(-1)), evaluate=False)
coveq

Eq(R**(-1)*C_1*R.T**(-1), R**(-1)*R*C_0*R.T*R.T**(-1))